In [1]:
from pyplasm import*
from larlib import*
import math
import csv

Evaluating fenvs.py..
...fenvs.py imported in 0.027388 seconds


In [2]:
def perimetro(): 
    with open("VillaCapra1.4.lines", "rb") as file:
        file_reader = csv.reader(file, delimiter=",")
        chiavi = {} ### diizionario delle chiavi
        nodi = [] ### array con i vertici
        archi= [] ### array con gli archi, ovvero gli spigoli
        i = 0 ### inizializzo un contatore
    
        for righe in file_reader:
            riga0 = math.floor(float(righe[0])*100)/100
            riga1 = math.floor(float(righe[1])*100)/100
            riga2 = math.floor(float(righe[2])*100)/100
            riga3 = math.floor(float(righe[3])*100)/100
        
            if not(riga0,riga1) in chiavi.keys():
                chiavi[(riga0,riga1)] = i
                nodi.append((riga0,riga1))
                i = i+1
            
            if not(riga2,riga3) in chiavi.keys():
                chiavi[(riga2,riga3)] = i
                nodi.append((riga2,riga3))
                i = i+1
            archi.append((chiavi[(riga0,riga1)],(chiavi[(riga2,riga3)])))
        pavimento = (CUBOID([850,600]))
        corpoVilla = MULTEXTRUDE(OFFSET([7,7])(STRUCT(MKPOLS([nodi,archi]))))(250)
    return corpoVilla

In [3]:
perimetro = (perimetro())

In [4]:
def columna(dm,h,h_base):
    
    cylndr = COMP([JOIN,TRUNCONE([dm/2,0.8*(dm/2),h])])(24)
    
    torus_bot = COMP([JOIN,TORUS([dm/12,dm/2])])([8,24])
    
    torus_top = COMP([JOIN,TORUS([0.8*(dm/12),0.8*(dm/2)])])([8,27])
    
    base = COMP([T([1,2])([7*(dm/-12),7*(dm/-23)]),CUBOID])([7*(dm/6),7*(dm/6),h_base])
    
    base_top = COMP([T([1,2])([7*(dm/-12),7*(dm/-12)]),CUBOID])([7*(dm/6),7*(dm/6),dm/6])
    
    capital = SUM([COMP([JOIN,TRUNCONE([0.8*dm/2, 1.2*dm/2,h/8])])(4),COMP([R([1,2])(PI/4),JOIN,TRUNCONE([0.8*dm/2,1.2*dm/2,
                                                                                                         h/8])])(4)]) 
    return (TOP([TOP([TOP([TOP([TOP([base, torus_bot]), cylndr]), torus_top]), capital]), base_top]))



In [5]:
def piramide(depth,width,height):
    
    _width = float(width)
    
    _scale = float(depth)/_width
    
    _height = float(height)
    
    return SCALE(2)(_scale)(ROTATE([1,2])(PI/4)(CONE([_width*(1/SQRT(2)),_height])(4)))

def elementoTriangolare():
    
    return R([1,2])(PI/2)((R([2,3])(-PI)(DIFFERENCE([CUBOID([154/2,154/2,60]),piramide(154,70000,60)]))))

In [253]:
def sferaDimezzata (radius):
        def GSPHERE0 (subds):
            N , M = subds
            domain = Hpc(Grid([N*[PI/N],M*[PI/M]]))
            domain = MAT([[1,0,0,0],[PI/2,1,0,0],[PI,0,1,0],[0,0,0,1]])(domain)
            fx  = lambda p: radius * math.cos(p[0]) * math.sin  (p[1])
            fy  = lambda p: radius * math.cos(p[0]) * math.cos (p[1])
            fz  = lambda p: radius * math.sin(p[0])
            ret=  GMAP([fx, fy, fz])(domain)
            return ret
        return GSPHERE0

In [254]:
def tettoTriangolare(depth,width):
    return TEXTURE("tetto.jpg")(S([1,2,3])([22,28.34,30])((ROTATE([1, 2])(PI)(ROTATE([2,3])(PI/2)(
        MULTEXTRUDE(MKPOL([[[0, 0],[width/2,width/4],[width,0]],[[1,2,3]],None]))(depth))))))

In [387]:
p2 = T([1,2])([-144,-420])(perimetro)
VIEW(p2)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f11084c2990> >

In [428]:
finestraFrnt1= T([1,2,3])([-10,20,60])(CUBOID([20,20,50]))
finestraFrnt2 = T([1,2,3])([-10,20,220])(CUBOID([20,20,20]))
finestraFrnt3 = T([1,2,3])([-10,80,220])(CUBOID([20,20,20]))
finestraLat1 = T([1,2,3])([20,-20,60])(CUBOID([20,20,50]))
finestraLat2 = T([1,2,3])([20,-20,220])(CUBOID([20,20,20]))
finestraLat3 = T([1,2,3])([80,-20,220])(CUBOID([20,20,20]))
finestraFrntLat = STRUCT([finestraFrnt1,finestraFrnt2,finestraFrnt3,finestraLat1,finestraLat2,finestraLat3])
perimetroFinestrato = DIFFERENCE([p2,finestraFrntLat])
VIEW(perimetroFinestrato)


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f11084857e0> >

In [429]:
verandaDx = CUBOID([154/3,143/2,10])

In [430]:
colonnatiVerandaDx = STRUCT([verandaDx,T([1,2,3])([14,-14/2,10])(columna(14,100,10)),
                            T([1,2,3])([14,143/2-14,10])(columna(14,100,10)),
                            T([1,2,3])([14,143/2-14*3-4,10])(columna(14,100,10))])

Child 3
Child 3
Child 3
Child 3
Child 3
Child 3


In [431]:
verandaSx = CUBOID([154/2,154/3,10])

In [432]:
colonnatiVerandaSx = STRUCT([verandaSx,T([2,3])([14/2,10])(columna(14,100,10)),
                            T([1,2,3])([14+14+14/2,14/2,10])(columna(14,100,10)),
                            T([1,2,3])([154/2-14/2,14/2,10])(columna(14,100,10))])

Child 3
Child 3
Child 3
Child 3
Child 3
Child 3


In [433]:
basamentoScala= DIFFERENCE([CUBOID([154,30,60]),T(1)(114)(CUBOID([30,30,50]))])

In [434]:
arcata = DIFFERENCE([CUBOID([154/3,10,195]),T(1)(10)(CUBOID([30,10,175]))])

In [435]:
scalinataDx=STRUCT([basamentoScala,S([1,2])([1.4,1.11])(T(3)(60)(elementoTriangolare())),T(1)(154-154/3)(arcata)])

In [436]:
#VIEW(scalinataDx)

In [437]:
scalinataSx=STRUCT([basamentoScala,S([1,2])([1.4,1.11])(T([2,3])([-60,60])(elementoTriangolare())),T([1,2])([154-154/3,30-10])(arcata)])

In [438]:
#VIEW(scalinataSx)

In [440]:
basamentoScalaDx = T([1,2])([-154,143/2-15])(STRUCT([scalinataDx,T([1,2,3])([154-154/3,15,50])(colonnatiVerandaDx)]))
#VIEW(basamentoScalaDx)

In [441]:
basamentoScalaSx = T([1,2])([30+154/2-15,-160])(STRUCT([T([1,2,3])([-15,154-154/3,50])(colonnatiVerandaSx),(R([1,2])(PI/2)(scalinataSx))]))

In [442]:
tettuccioSx = STRUCT([T([1,2,3])([-50,58,195])(CUBOID([55,88,10])),T([1,2,3])([4,51.5,252])((R([1,2])(PI/2)(S([1,2,3])([1.2,0.7,0.8])(elementoTriangolare()))))])

In [443]:
tettuccioDx = STRUCT([R([1,2])(PI)(T([1,2,3])([-150.5,2,195])(CUBOID([87,55,10]))),(T([1,2,3])([60,-57,253])(R([1,2])(PI)(R([1,2])(PI)(S([1,2,3])([1.2,0.7,0.8])(elementoTriangolare())))))])

In [444]:
tettox = INTERSECTION([tettoTriangolare1(10,20),T([1,2])([-80,360])(R([1,2])(PI/2)(tettoTriangolare(10,20)))])
tetto = DIFFERENCE([tettox,T(1)(-365)(CUBOID([290,350,50]))])

In [445]:
spessore = CUBOID([0.1,0.1,0.1])
sferaDimezzata1 = (sferaDimezzata(1.1)([30,30]))
sferaDimezzataX = JOIN([sferaDimezzata1,spessore])

Child 3


In [446]:
primoAnello = T(3)(1)(CYLINDER([0.7,0.1])(30))
secondoAnello = T(3)(0.8)(CYLINDER([0.9,0.1])(30))
terzoAnello = T(3)(0.6)(CYLINDER([1,0.1])(30))
quartoAnello = T(3)(0.4)(CYLINDER([1.1,0.1])(30))
fregioCircolare = T(3)(0.2)(CYLINDER([1.2,0.1])(30))
sfera = R([1,2])(-PI/4)(R([1,3])(PI/2)((sferaDimezzataX)))
cilindro =CYLINDER([1.1,0.3])(30)
cilindro1 = T([1,2,3])([-0.15,-0.2,0.1])(CUBOID([0.3,0.4,0.15]))
cilindro2 = CYLINDER([0.2,0.3])(30)
cilindroTop=DIFFERENCE([cilindro2,cilindro1])
cupola = TOP([STRUCT([cilindro,T(3)(0.2)(sfera),primoAnello,fregioCircolare,secondoAnello,
                    terzoAnello,quartoAnello]),cilindroTop])
VIEW(cupola)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f1108531120> >

In [447]:
tettoECupola=STRUCT([tetto,T([1,2,3])([-225,140,110])(S([1,2,3])([70,70,70])(cupola))])

In [448]:
VIEW(tettoECupola)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f1108485c90> >

In [449]:
corpoVilla14 = STRUCT([perimetroFinestrato,basamentoScalaDx,basamentoScalaSx,tettuccioSx,tettuccioDx])

In [450]:
VIEW(corpoVilla14)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f1108531840> >

In [451]:
corpoVilla=STRUCT([corpoVilla14,T([1,2])([154*2-12,-6.5])(R([1,2])(PI/2)(corpoVilla14))])

In [452]:
VIEW(corpoVilla)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f11085318d0> >

In [453]:
VillaCapra = STRUCT([corpoVilla,T([1,2])([154*2-5,290])(R([1,2])(PI)(corpoVilla)),T([1,2,3])([415,-23,200])(S([1,2])([1.2,1.2])(tettoECupola))])

In [455]:
VIEW(VillaCapra)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f1108531360> >